 
Для произвольно выбранного датасета провести обработку данных и построить предсказательную модель с использованием функционала pySpark.
 

V1
28.02.24
Anatoly

Для примера я взял датасет с титпником и решение с случайным лесом но слегка его изменил добавив метрики 

In [96]:
import numpy as np
import pandas as pd
import spacy 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
nlp = spacy.load("ru_core_news_sm") 
%matplotlib inline
import seaborn as sns; sns.set()



file_path_test ="C:/Users/qiarr/Desktop/task/lesson 18/titanic_test.csv"
file_path_train = "C:/Users/qiarr/Desktop/task/lesson 18/titanic_train.csv"
df_test = pd.read_csv(file_path_test, encoding='cp1251', )
df_train = pd.read_csv(file_path_train, encoding='cp1251',)


In [97]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [98]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [99]:


df_train_cleaned = df_train.drop(['Cabin', 'Embarked',"PassengerId","Name","Ticket"], axis=1)# метод drop для удаления указанных столбцов

df_train_cleaned.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [100]:
for i in df_train_cleaned.columns:
    unic = df_train_cleaned[i].unique()
    print(f"Столбец {i}: {unic}")

Столбец Survived: [0 1]
Столбец Pclass: [3 1 2]
Столбец Sex: ['male' 'female']
Столбец Age: [22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.  ]
Столбец SibSp: [1 0 3 4 2 5 8]
Столбец Parch: [0 1 2 5 3 4 6]
Столбец Fare: [  7.25    71.2833   7.925   53.1      8.05     8.4583  51.8625  21.075
  11.1333  30.0708  16.7     26.55    31.275    7.8542  16.      29.125
  13.      18.       7.225   26.       8.0292  35.5     31.3875 263.
   7.8792   7.8958  27.7208 146.5208   7.75    10.5     82.1708  52.
   7.2292  11.2417   

In [101]:
nan_finder = df_train_cleaned["Fare"].isna().any()
print(f"В столбце есть NaN: {nan_finder}")
# Данных много и невсем понятно есть ли данные с nan в цене билета

В столбце есть NaN: False


Исходя из наблюдений мы видм что в двтасете есть данные NAN в AGE ,которые не очень хотелось бы терять,подставим медиану вместо них

In [102]:
median = df_train_cleaned["Age"].median()
df_train_cleaned["Age"] = df_train_cleaned["Age"].fillna(median)
print(df_train_cleaned["Age"].unique()) 

[22.   38.   26.   35.   28.   54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.    8.   19.   40.   66.   42.   21.   18.
  3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.   16.
 25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.   14.5
 70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.   56.
 50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.   60.
 10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.    0.67
 30.5   0.42 34.5  74.  ]


In [103]:
from sklearn.model_selection import train_test_split
X = pd.get_dummies(df_train_cleaned.drop(columns=["Survived"]))  # Замените "target_column" на название вашего целевого столбца
y = df_train_cleaned["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)






In [104]:
X

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
0,3,22.0,1,0,7.2500,False,True
1,1,38.0,1,0,71.2833,True,False
2,3,26.0,0,0,7.9250,True,False
3,1,35.0,1,0,53.1000,True,False
4,3,35.0,0,0,8.0500,False,True
...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,False,True
887,1,19.0,0,0,30.0000,True,False
888,3,28.0,1,2,23.4500,True,False
889,1,26.0,0,0,30.0000,False,True


In [105]:


model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
model.fit(X_train,y_train)
predictions = model.predict(X_test)

In [106]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Вывод метрик
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8044692737430168
Precision: 0.8
Recall: 0.7027027027027027
F1 Score: 0.7482014388489208


In [107]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, median,when

# Инициализируем SparkSession
spark = SparkSession.builder \
    .appName("DataFrameExample") \
    .getOrCreate()
file_path = "C:/Users/qiarr/Desktop/task/lesson 23/task/titanic_train.csv"
df_train_spark = spark.read.csv(file_path, header=True, inferSchema=True)

# Удаляем указанные столбцы
columns_to_drop = ['Cabin', 'Embarked', 'PassengerId', 'Name', 'Ticket']
df_train_cleaned_spark = df_train_spark.drop(*columns_to_drop)

# Вычисляем медиану для столбца 'Age'
age_median = df_train_cleaned_spark.select(median(col('Age'))).collect()[0][0]

# Заменяем пропущенные значения в столбце 'Age' на медиану
df_train_cleaned_spark = df_train_cleaned_spark.withColumn('Age',
                                               when(col('Age').isNull(), age_median)
                                               .otherwise(col('Age')))



In [108]:
df_train_cleaned_spark.show()


+--------+------+------+----+-----+-----+-------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|
+--------+------+------+----+-----+-----+-------+
|       0|     3|  male|22.0|    1|    0|   7.25|
|       1|     1|female|38.0|    1|    0|71.2833|
|       1|     3|female|26.0|    0|    0|  7.925|
|       1|     1|female|35.0|    1|    0|   53.1|
|       0|     3|  male|35.0|    0|    0|   8.05|
|       0|     3|  male|28.0|    0|    0| 8.4583|
|       0|     1|  male|54.0|    0|    0|51.8625|
|       0|     3|  male| 2.0|    3|    1| 21.075|
|       1|     3|female|27.0|    0|    2|11.1333|
|       1|     2|female|14.0|    1|    0|30.0708|
|       1|     3|female| 4.0|    1|    1|   16.7|
|       1|     1|female|58.0|    0|    0|  26.55|
|       0|     3|  male|20.0|    0|    0|   8.05|
|       0|     3|  male|39.0|    1|    5| 31.275|
|       0|     3|female|14.0|    0|    0| 7.8542|
|       1|     2|female|55.0|    0|    0|   16.0|
|       0|     3|  male| 2.0|    4|    1| 29.125|


In [109]:
from pyspark.sql.functions import when

# Создаем новый столбец 'Sex_numeric', где 'female' будет 1, а 'male' - 2
df_train_cleaned_spark = df_train_cleaned_spark.withColumn('Sex_numeric',
                                           when(df_train_cleaned_spark['Sex'] == 'female', 1)
                                           .otherwise(2))

# Удаляем столбец 'Sex' из DataFrame
df_train_cleaned_spark = df_train_cleaned_spark.drop("Sex")

In [110]:
df_train_cleaned_spark.show()

+--------+------+----+-----+-----+-------+-----------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Sex_numeric|
+--------+------+----+-----+-----+-------+-----------+
|       0|     3|22.0|    1|    0|   7.25|          2|
|       1|     1|38.0|    1|    0|71.2833|          1|
|       1|     3|26.0|    0|    0|  7.925|          1|
|       1|     1|35.0|    1|    0|   53.1|          1|
|       0|     3|35.0|    0|    0|   8.05|          2|
|       0|     3|28.0|    0|    0| 8.4583|          2|
|       0|     1|54.0|    0|    0|51.8625|          2|
|       0|     3| 2.0|    3|    1| 21.075|          2|
|       1|     3|27.0|    0|    2|11.1333|          1|
|       1|     2|14.0|    1|    0|30.0708|          1|
|       1|     3| 4.0|    1|    1|   16.7|          1|
|       1|     1|58.0|    0|    0|  26.55|          1|
|       0|     3|20.0|    0|    0|   8.05|          2|
|       0|     3|39.0|    1|    5| 31.275|          2|
|       0|     3|14.0|    0|    0| 7.8542|          1|
|       1|

In [111]:
from pyspark.ml.feature import VectorAssembler

# Список признаков, которые вы хотите объединить в вектор
input_features = ["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex_numeric"]

# Инициализация VectorAssembler для объединения признаков в векторы
assembler = VectorAssembler(
    inputCols=input_features,
    outputCol="features"
)

# Преобразование DataFrame, добавляя столбец 'features'
df_train_cleaned_spark_features = assembler.transform(df_train_cleaned_spark)

In [112]:
train_data, test_data = df_train_cleaned_spark_features.randomSplit([0.8, 0.2], seed=42)

X_train_spark = train_data.drop("Survived")
y_train_spark = train_data.select("Survived")
X_test_spark = test_data.drop("Survived")
y_test_spark = test_data.select("Survived")

In [113]:


# Проверка наличия столбца 'features' в новом DataFrame
df_train_cleaned_spark_features.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Sex_numeric: integer (nullable = false)
 |-- features: vector (nullable = true)



In [116]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Создаем объект MulticlassClassificationEvaluator для вычисления precision, recall и f1-score
evaluator = MulticlassClassificationEvaluator(
    labelCol="Survived", 
    predictionCol="prediction", 
    metricName="weightedPrecision"  # можно использовать также "weightedRecall" и "f1"
)
# Инициализация модели RandomForestClassifier
rf = RandomForestClassifier(
    numTrees=100,
    maxDepth=3,
    seed=42,
    labelCol="Survived",  # Указываем столбец 'Survived' как целевую переменную
    featuresCol="features"
)




# Обучение модели на тренировочных данных
model = rf.fit(train_data)

# Предсказание на тестовых данных
predictions = model.transform(test_data)

# Оценка качества модели
precision = evaluator.evaluate(predictions)

# Меняем метрику на recall
evaluator.setMetricName("weightedRecall")
recall = evaluator.evaluate(predictions)

# Меняем метрику на f1-score
evaluator.setMetricName("f1")
f1 = evaluator.evaluate(predictions)
accuracy = evaluator.evaluate(predictions)

# Выводим метрику качества модели
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.8083726483625048
Precision: 0.822243423742674
Recall: 0.8137931034482758
F1-score: 0.8083726483625048
